In [2]:
import pandas as pd
import numpy as np
# train data를 df로
df_train = pd.read_excel(r'.\projects\wannaone\remixed.xlsx')
# data.frame을 NumPy 배열로
# train set
text_train = df_train['contents'].as_matrix()
y_train = df_train['label'].as_matrix()

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression as lgt

# tokeinize - Twitter
from konlpy.tag import Twitter
twit = Twitter()
words = ['Verb','Adjective','Punctuation','Adverb','Conjunction','Exclamation','KoreanParticle']
def tokenize(doc):
    tagged = ['/'.join(t) for t in twit.pos(doc, norm=True, stem=True)]
    result = [word for word in tagged if word.split('/')[-1] in words]
    return result
tfidf = TfidfVectorizer(tokenizer=tokenize,ngram_range=(1,3),min_df=3)

In [1]:
from konlpy.tag import Komoran
k = Komoran ()

In [2]:
k.pos('ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 존아 실시간으로 바뀌는 인간 감정 다큐멘터리 한 편 찍으셔따.. ㅅㅂ 아 오늘 워너원 화보 촬영 들어갔대..')

[('ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 'NA'),
 ('존', 'NNP'),
 ('아', 'JKV'),
 ('실시간', 'NNG'),
 ('으로', 'JKB'),
 ('바뀌', 'VV'),
 ('는', 'ETM'),
 ('인간', 'NNG'),
 ('감정', 'NNG'),
 ('다큐멘터리', 'NNG'),
 ('한', 'MM'),
 ('편', 'NNB'),
 ('찍', 'MAG'),
 ('으', 'NNG'),
 ('시', 'VA'),
 ('어', 'EC'),
 ('따', 'VV'),
 ('아', 'EF'),
 ('.', 'SF'),
 ('.', 'SF'),
 ('ㅅㅂ', 'NA'),
 ('아', 'IC'),
 ('오늘', 'NNG'),
 ('워너', 'NNP'),
 ('원', 'NNB'),
 ('화보', 'NNG'),
 ('촬영', 'NNG'),
 ('들어가', 'VV'),
 ('았', 'EP'),
 ('대', 'EF'),
 ('.', 'SF'),
 ('.', 'SF')]

In [22]:
tokenize('워너원 애들.... 솔직히 내가 2주만 지나도 다 적응할 것을 알고 잇다')

['..../Punctuation',
 '솔직하다/Adjective',
 '지나다/Verb',
 '다/Adverb',
 '하다/Verb',
 '잇다/Verb']

In [25]:
tokenize('ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 존아 실시간으로 바뀌는 인간 감정 다큐멘터리 한 편 찍으셔따.. ㅅㅂ 아 오늘 워너원 화보 촬영 들어갔대..')

['ㅋㅋ/KoreanParticle',
 '바뀌다/Verb',
 '하다/Verb',
 '찍다/Verb',
 '따다/Verb',
 '../Punctuation',
 'ㅅㅂ/KoreanParticle',
 '아/Exclamation',
 '들어가다/Verb',
 '../Punctuation']

In [23]:
tokenize('워너원애들이 화장품 팡팡하며 광고찍는거 생각하니까 갑자기 기분좋아져따')

['팡팡/Adverb', '하다/Verb', '찍다/Verb', '하다/Verb', '좋아지다/Adjective', '따다/Verb']

In [12]:
input_data = tfidf.fit_transform(text_train.astype('U'))
clf_1 = lgt(C=10)
print('ML Start')
model = clf_1.fit(input_data,y_train)
print('ML Finish')

ML Start
ML Finish


In [13]:
model.score(input_data,y_train)

0.84156694966241063

In [14]:
# read_data
df_1 = pd.read_csv(r'.\projects\wannaone\crawled_data18.txt',
                   delimiter='\t',keep_default_na=False,header=None)

In [15]:
h_x = df_1[3].as_matrix()

In [16]:
#wanna_one
input_test = tfidf.transform(h_x)
result = model.predict(input_test)
result_p = model.predict_proba(input_test)

In [17]:
df_prob = pd.DataFrame(result_p)
result_p0 = df_prob[0]
result_p1 = df_prob[1]
point = round((result_p1 - result_p0),2)*100

data = {'contents':h_x,
       'predict_label':result,
       'point':point,
       'neg':result_p0,
       'pos':result_p1}
data_pandas = pd.DataFrame(data)
from IPython.display import display
display(data_pandas)

,contents,neg,point,pos,predict_label
0,오늘 친구가 워너원 봤대요ㅠ ㅠ ㅠ ㅠ엉엉 같이 함께 뭐 먹었나봐여ㅠㅠㅠㅠㅠㅠㅠㅠㅠ...,0.061335,88.0,0.938665,1
1,"올해 가장 행복한 순간 TOP 3를 꼽자면 널 처음 알게된 순간, 널 처음 본 순간...",0.102175,80.0,0.897825,1
2,워너원 망했다고 광광댔던 사람들 우는 소리 들린다,0.540617,-8.0,0.459383,0
3,"[단독] '프듀2' 워너원 11人, 합숙 시작 ＂첫 스케줄은 화보 촬영＂ (출처 :...",0.073223,85.0,0.926777,1
4,우리 배챙이 진영이의 워너원 데뷔를 축하하며 소소한 이벤트를 진행합니다. 계정팔로+...,0.321872,36.0,0.678128,1
5,170616 대휘 안우러 #이대휘 #대휘 #강다니엘 #워너원,0.671254,-34.0,0.328746,0
6,워너원 트위터 헤더 ・:*:・゜ 저장하실 때는 리트윗 해 주세요 재가공 재배...,0.503231,-1.0,0.496769,0
7,"[단독] '프듀2' 워너원 11人, 합숙 시작 ＂첫 스케줄은 화보 촬영＂ (출처 :...",0.073223,85.0,0.926777,1
8,워너원 화보 촬영 성운이 화보.......,0.565017,-13.0,0.434983,0
9,"'프듀2' 데뷔팀 워너원, 자연주의 화장품 CF 모델 발탁 '대세 예약'",0.461167,8.0,0.538833,1


In [18]:
data_pandas.to_excel("result5.xlsx",encoding='utf-8')